ABAG Landuse Calculations for VTA and BART Model.

This notebook documents the joining data between VTA or CCAG TAZ Zones and MTC's 1454 regions.  VTA and CCAG will be reffered to as Regional Partners. 

Also there is details on how to convert ABAG numbers to BART numbers.

The inputs are the yearly calculations from MTC and VTA and CCAG.  The outputs are a csv, excel, and a shapefile.

In [22]:
#This uses geopy36
import geopandas as gpd
import pandas as pd
%matplotlib inline  
from shapely.geometry import Point

from simpledbf import Dbf5
#pip install simpledbf
#https://pypi.python.org/pypi/simpledbf/0.2.4

In [23]:
# RP stands for Regional Partner
def prep_data(abag, rp_taz, rel, geom_given = True):
    
    cols = ['TAZ','DIST','SDIST','COUNTY','TOTHH','TOTPOP',
                                    'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
                                    'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
                                    'AGE2044','AGE4564','AGE65','AGE0513','AGE1417','AGE1824','SFHH','MFHH']
    cols_to_retain = ['INC1','INC2','INC3','INC4','MHHINC', 'ESENR', 'HSENR', 'COLLENR', 'COLLENRF', 'COLLENRP']

    cols.extend(cols_to_retain)

    if geom_given:
        cols.append('geometry')
    else:
        pass
    
    
    rp_taz = rp_taz[cols].rename(columns={"TOTHH":"RP_TOTHH","TOTPOP":"RP_TOTPOP",
                                "HHPOP":"RP_HHPOP","EMPRES":"RP_EMPRES","RESACRE":'RP_RESACRE',"CIACRE":"RP_CIACRE","TEMP":"RP_TEMP"})
    # rp_taz.rename(columns='RP_{}'.format)

    #Join the RP shapefile to the abag 2010 dataset!
    rp_calc = pd.merge(pd.merge(abag[['TAZ1454','TOTHH','RESACRE','CIACRE','TOTEMP','SHPOP62P']].rename(columns={"SHPOP62P":"Z2SHARE"}),
                                rel,how='outer'),rp_taz)

    return rp_calc
    # vt_calc.groupby(['TAZ1454','TAZ'])['VTA_TOTPOP'].apply(lambda x: x / x.sum())
    # vt_calc.groupby(['TAZ1454','TAZ'])['VTA_TOTPOP'].sum()

In [24]:
def landuse_calcs(rp_calc):
    """ Takes the input dataframe and does transformations, share calculations, to derive numbers for projections.
    """
    rp_tothh = rp_calc.groupby(['TAZ1454','TAZ'])['RP_TOTHH'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

    rp_resacre = rp_calc.groupby(['TAZ1454','TAZ'])['RP_RESACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_ciacre = rp_calc.groupby(['TAZ1454','TAZ'])['RP_CIACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_temp = rp_calc.groupby(['TAZ1454','TAZ'])['RP_TEMP'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

    rp_tothh = rp_tothh.rename(columns={"RP_TOTHH":"RP_TOTHH_share"})

    rp_resacre = rp_resacre.rename(columns={"RP_RESACRE":"RP_RESACRE_share"})
    rp_ciacre = rp_ciacre.rename(columns={"RP_CIACRE":"RP_CIACRE_share"})
    rp_temp = rp_temp.rename(columns={"RP_TEMP":"RP_TEMP_share"})

    vta_final= pd.merge(pd.merge(pd.merge(pd.merge(rp_calc,rp_tothh),rp_resacre),rp_ciacre),rp_temp)
    vta_final['abag_TOTHH_dist'] = round(vta_final['TOTHH']*vta_final['RP_TOTHH_share'])

    vta_final['abag_RESACRE_dist'] = round(vta_final['RESACRE']*vta_final['RP_RESACRE_share'])
    vta_final['abag_CIACRE_dist'] = round(vta_final['CIACRE']*vta_final['RP_CIACRE_share'])
    vta_final['abag_TEMP_dist'] = round(vta_final['TOTEMP']*vta_final['RP_TEMP_share'])

    vta_final['abag_HHPOP_dist'] = round(vta_final['RP_HHPOP']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])

    vta_final['abag_TOTPOP_dist'] = round(vta_final['RP_TOTPOP']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])

    vta_final['abag_EMPRES_dist'] = round(vta_final['RP_EMPRES']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])

    vta_final['abag_HH1_dist'] = round(vta_final['HH1']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    vta_final['abag_HH2_dist'] = round(vta_final['HH2']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    vta_final['abag_HH3_dist'] = round(vta_final['HH3']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    vta_final['abag_HH4_dist'] = round(vta_final['HH4']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    #Make HH

    vta_final['abag_RETEMP_dist'] = round(vta_final['RETEMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])
    vta_final['abag_SEREMP_dist'] = round(vta_final['SEREMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])
    vta_final['abag_OTHEMP_dist'] = round(vta_final['OTHEMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])
    vta_final['abag_AGEMP_dist'] = round(vta_final['AGEMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])
    vta_final['abag_MANEMP_dist'] = round(vta_final['MANEMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])
    vta_final['abag_WHOEMP_dist'] = round(vta_final['WHOEMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])

    vta_final['abag_AGE0004_dist'] = round(vta_final['AGE0004']/vta_final['RP_HHPOP']*vta_final['abag_HHPOP_dist'])
    vta_final['abag_AGE0519_dist'] = round(vta_final['AGE0519']/vta_final['RP_HHPOP']*vta_final['abag_HHPOP_dist'])
    vta_final['abag_AGE2044_dist'] = round(vta_final['AGE2044']/vta_final['RP_HHPOP']*vta_final['abag_HHPOP_dist'])
    vta_final['abag_AGE4564_dist'] = round(vta_final['AGE4564']/vta_final['RP_HHPOP']*vta_final['abag_HHPOP_dist'])
    vta_final['abag_AGE65_dist'] = round(vta_final['AGE65']/vta_final['RP_HHPOP']*vta_final['abag_HHPOP_dist'])

    vta_final['abag_AGE0513_dist'] = round(vta_final['AGE0513']/vta_final['RP_TOTPOP']*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE1417_dist'] = round(vta_final['AGE1417']/vta_final['RP_TOTPOP']*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE1824_dist'] = round(vta_final['AGE1824']/vta_final['RP_TOTPOP']*vta_final['abag_TOTPOP_dist'])
    
    vta_final['abag_SFHH_dist'] = round(vta_final['SFHH']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    vta_final['abag_MFHH_dist'] = round(vta_final['MFHH']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    
    return vta_final

In [5]:
# Prepping the MTC data
mtc_taz = gpd.read_file('MTC/')
abag = pd.read_csv('ABAG_03202018/run7224c_taz_summaries_2015.csv')
abag = abag.rename(columns={'ZONE':'TAZ1454'})
mtc_taz = pd.merge(mtc_taz,abag)

mtc_taz.head()

,STFID,FIPSSTCO,TRACT2,TRACT,TRACTID,SUPERD,TAZ1454,AREALAND,AREAWATR,LANDACRE,...,HHPOP,TOTPOP,EMPRES,AGE0004,AGE0519,AGE2044,AGE4564,AGE65P,total_job_spaces,total_residential_units
0,06001400100,06001,400100,400100,4001,19,1005,6799198,0,1680.118487,...,3027.0,3029.0,2086.0,119.0,249.0,799.0,1312.0,550.0,3344.0,1399.0
1,06001400200,06001,400200,400200,4002,19,999,659615,0,162.994423,...,1928.0,2002.0,1403.0,92.0,179.0,833.0,674.0,224.0,6134.0,909.0
2,06001400300,06001,400300,400300,4003,19,998,1074640,0,265.549338,...,5284.0,5325.0,3906.0,282.0,444.0,2592.0,1473.0,534.0,2087.0,2754.0
3,06001400400,06001,400400,400400,4004,19,1000,696057,0,171.999438,...,3711.0,3746.0,2609.0,175.0,337.0,1918.0,977.0,339.0,1434.0,1862.0
4,06001400500,06001,400500,400500,4005,19,1001,576343,0,142.417463,...,3423.0,3556.0,2280.0,171.0,430.0,1737.0,815.0,403.0,898.0,1662.0


In [ ]:
#HHINCQ1 - number of houses in the 1st quartile.
mtc_taz.columns

In [ ]:
mtc_taz.columns

In [ ]:
# Prep the VTA data

vta_taz = gpd.read_file('VTA_TAZ/')
# vta_taz = vta_taz.rename(columns={'TAZ':'VTA_TAZ'})

dbf = Dbf5('2017ABAGLanduseAllocation/VTA/zmast13.dbf')
vta_dbf = dbf.to_dataframe()
vta_dbf = vta_dbf.rename(columns={'ZONE':'TAZ'})
vta_taz = pd.merge(vta_taz,vta_dbf)


# mtc_taz = gpd.read_file('/Users/vivek/Github/VTA/Landuse/MTC/MTCTAZ1454.dbf')


#http://analytics.mtc.ca.gov/foswiki/UrbanSimTwo/OutputToTravelModel

# vta_taz['centroid'] = vta_taz.centroid
# vta_taz = vta_taz.set_geometry('centroid')
# vta_taz['old_geometry'] = vta_taz['geometry']
# vta_taz['geometry'] = vta_taz['centroid']

# centroid = gpd.sjoin(mtc_taz[['TAZ1454','geometry']], vta_taz, how = "right", op='contains')
# centroid['TAZ1454'].nunique()
# centroid[['TAZ1454','TAZ']].to_csv("rel.csv")
# rel = centroid[['TAZ1454','TAZ']]
# missing_zones = pd.DataFrame([{'TAZ1454' : 1454, 'TAZ' : 2786},{'TAZ1454':404,'TAZ':980},{'TAZ1454' : 190, 'TAZ' : 1890}])
# rel = pd.concat([rel,missing_zones])
# rel.to_csv('rel_vta_mtc.csv',index=False)

rel = pd.read_csv('rel_vta_mtc.csv')
rel = rel.loc[~rel['TAZ1454'].isnull(),]

vta_taz = pd.merge(vta_taz[['TAZ','DIST','SDIST','COUNTY','geometry','TOTHH','TOTPOP',
                            'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
                            'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
                            'AGE2044','AGE4564','AGE65','SFHH','MFHH']],rel)

In [ ]:
# Prep Data for CCAG

CCAG_taz = gpd.read_file('CCAGTAZ/')

# Not sure what the data in the TAZ shapefile relates to, will delete.
del CCAG_taz['ESENR']
del CCAG_taz['HSENR']
# CCAG_taz.rename(columns={'CITY':'CITY_NAME'}, inplace=True)


dbf = Dbf5('2017ABAGLanduseAllocation/CCAGP2013/zmast13.dbf')
CCAG_dbf = dbf.to_dataframe()
# vta_dbf = vta_dbf.rename(columns={'ZONE':'VTA_TAZ'})


ccag_merged = pd.merge(CCAG_taz, CCAG_dbf, left_on = ['TAZ','DIST','COUNTY','SDIST'], right_on=['ZONE','DIST','COUNTY','SDIST'])
del ccag_merged['ZONE']

In [39]:
#Prep ABAG data
#BART: Update abag projection to new file.

mtc_taz = gpd.read_file('MTC/')
# mtc_taz = mtc_taz.rename(columns={'TAZ1454':'ZONE'})
years = ['2015','2020','2025','2030','2035','2040']


#Prep BART data

bart_dbf = pd.read_excel('bart_calcs_clean2015_AvdH_corrected.xlsx', sheet_name='Corrected')

# bart_dbf = bart_dbf.rename(columns={'BARTZONE':'TAZ'})
# year = '2015'

# # bart_dbf = pd.read_excel('BART_2025_2035_clean_landuse_data.xlsx', sheet_name='ZMAST25')
# bart_dbf = pd.read_excel('bart_calcs_clean2015_AvdH_for_Vivek.xlsx', sheet_name='Corrected')
# bart_dbf = bart_dbf.rename(columns={'ZONE':'TAZ'})
# year = '2025'

# bart_dbf = pd.read_excel('bart_calcs_clean2015_AvdH_for_Vivek.xlsx', sheet_name='Corrected')
# # bart_dbf = pd.read_excel('BART_2025_2035_clean_landuse_data.xlsx', sheet_name='ZMAST35')
# bart_dbf = bart_dbf.rename(columns={'ZONE':'TAZ'})
# year = '2035'

# bart_dbf = pd.read_excel('BART_2025_2035_clean_landuse_data.xlsx', sheet_name='ZMAST35')
# bart_dbf = bart_dbf.rename(columns={'ZONE':'TAZ'})
year = '2040'


# pd.read_excel?
bart_rel = pd.read_excel('BART_zmast15_EIR.xlsx', sheet_name='Zone Corresp')
bart_rel = bart_rel.rename(columns={'BARTZONE':'TAZ', 'MTCTAZ':'TAZ1454'})

bart_dbf = pd.merge(bart_dbf, bart_rel)

# This might be inefficient since the relationship seems to be inside the bart_TAZ shapefile.
bart_taz = gpd.read_file('BART-TAZ/TAZ/')
bart_crs = bart_taz.crs
bart_taz = pd.merge(bart_taz[['TAZ','KEY','AREA', 'geometry']],bart_dbf)

# vta_taz[['TAZ','DIST','SDIST','CITY','COUNTY','geometry','TOTHH','TOTPOP',
#                             'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
#                             'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
#                             'AGE2044','AGE4564','AGE65','SFHH','MFHH']]




abag = pd.read_csv('ABAG_03202018/run7224c_taz_summaries_' + year + '.csv')
abag = abag.rename(columns={'ZONE':'TAZ1454'})
mtc_taz = pd.merge(mtc_taz,abag)

rp_calc = prep_data(abag, bart_taz, bart_rel, geom_given = True)

In [40]:
bart_calcs = landuse_calcs(rp_calc)
del bart_calcs['geometry']
bart_calcs.to_csv('output/bart_proj_abag_calcs_' + year + '.csv',index=False)
bart_calcs.to_excel('output/bart_proj_abag_calcs_'  + year + '.xlsx' ,index=False)

bart_calcs_final = bart_calcs[['TAZ', 'DIST', 'SDIST', 'COUNTY', 'abag_TOTHH_dist', 'abag_HHPOP_dist',
       'abag_TOTPOP_dist', 'abag_EMPRES_dist', 'abag_SFHH_dist', 'abag_MFHH_dist', 'abag_HH1_dist', 'abag_HH2_dist', 'abag_HH3_dist', 'abag_HH4_dist',
       'INC1', 'INC2', 'INC3', 'INC4', 'MHHINC', 'TACRES', 'abag_RESACRE_dist',
       'abag_CIACRE_dist', 'Z2SHARE', 'abag_TEMP_dist', 'abag_RETEMP_dist', 'abag_SEREMP_dist', 'abag_OTHEMP_dist', 'abag_AGEMP_dist',
       'abag_MANEMP_dist', 'abag_WHOEMP_dist', 'abag_AGE0004_dist', 'abag_AGE0519_dist', 'abag_AGE2044_dist', 'abag_AGE4564_dist',
       'abag_AGE65_dist', 'abag_AGE0513_dist', 'abag_AGE1417_dist', 'abag_AGE1824_dist', 'ESENR', 'HSENR',
       'COLLENR', 'COLLENRF', 'COLLENRP','TAZ1454']].rename(columns={'TAZ':'ZONE'})

# df.rename(columns=lambda x: x + 1)
# https://www.dataquest.io/blog/pandas-cheat-sheet/
bart_calcs_final.rename(columns=lambda x: x.replace('abag_','').replace('_dist',''),inplace=True)

#     crs = mtc_taz.crs
# geo_df = gpd.GeoDataFrame(bart_final, crs=bart_crs, geometry=bart_final['geometry'])

# geo_df.head()
# geo_df.to_file('output/bart_proj_abag_' + year)

# del bart_calcs_final['geometry']


bart_calcs_final.to_csv('output/bart_calcs_clean' + year + '.csv',index=False)
bart_calcs_final.to_excel('output/bart_calcs_clean'  + year + '.xlsx' ,index=False)

In [ ]:
# Generate CCAG Calculations

rp_calc = prep_data(abag, ccag_merged, rel)
vta_final = landuse_calcs(rp_calc)

geometry = vta_final['geometry']
crs = mtc_taz.crs
geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)

geo_df.head()
geo_df.to_file('abag_2010_ccag_dist')

del vta_final['geometry']
vta_final.to_csv('ccag_final_abag_2010.csv',index=False)
vta_final.to_excel('ccag_final_abag_2010.xlsx',index=False)

In [ ]:
# Generate VTA Calculations

rp_calc = prep_data(abag, vta_taz)
vta_final = landuse_calcs(rp_calc)

geometry = vta_final['geometry']
crs = mtc_taz.crs
geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)


geo_df.head()
geo_df.to_file('abag_2010_vta_dist')

del vta_final['geometry']
vta_final.to_csv('vta_final_abag_2010.csv',index=False)
vta_final.to_excel('vta_final_abag_2010.xlsx',index=False)

In [ ]:
# # Generate BART Calculations

# rp_calc = prep_data(abag, vta_taz)
# vta_final = landuse_calcs(rp_calc)

# geometry = vta_final['geometry']
# crs = mtc_taz.crs
# geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)


# geo_df.head()
# geo_df.to_file('abag_2010_vta_dist')

# del vta_final['geometry']
# vta_final.to_csv('vta_final_abag_2010.csv',index=False)
# vta_final.to_excel('vta_final_abag_2010.xlsx',index=False)

In [ ]:
# vta_final.head()

In [ ]:
# vt = vta_taz.columns.values
# ab = abag_2010.columns.values 
# set(ab) - set(tz)
# # set(ab) - set(tz)
# # set(ab).difference(set(tz))
# set(vt).intersection(set(ab))
# set(ab) - set(vt)